<a href="https://colab.research.google.com/github/nusrat-cste/AI/blob/master/Human_Activity_Recognition_Using_Wearable_Sensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PAMAP2 dataset

# Loading script

In [1]:
import pandas as pd
import numpy as np
import os
from google.colab import files
from numpy.lib.stride_tricks import sliding_window_view


In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Assigning column names

In [3]:
col_names = ['ts','activityID','hr']+[f'IMU_hand_{x+1}' for x in range(17)]+[f'IMU_chest_{x+1}' for x in range(17)]+[f'IMU_ankle_{x+1}' for x in range(17)]
len(col_names)

54

## File upload

In [4]:
# file upload from drive for faster access
time101 = pd.read_csv('/content/drive/MyDrive/timedata/subject101.zip',sep = " ", names = col_names)
time102 = pd.read_csv('/content/drive/MyDrive/timedata/subject102.zip',sep = " ", names = col_names)
time103 = pd.read_csv('/content/drive/MyDrive/timedata/subject103.zip',sep = " ", names = col_names)
time104 = pd.read_csv('/content/drive/MyDrive/timedata/subject104.zip',sep = " ", names = col_names)
time105 = pd.read_csv('/content/drive/MyDrive/timedata/subject105.zip',sep = " ", names = col_names)
time106 = pd.read_csv('/content/drive/MyDrive/timedata/subject106.zip',sep = " ", names = col_names)
time107 = pd.read_csv('/content/drive/MyDrive/timedata/subject107.zip',sep = " ", names = col_names)
time108 = pd.read_csv('/content/drive/MyDrive/timedata/subject108.zip',sep = " ", names = col_names)
time109 = pd.read_csv('/content/drive/MyDrive/timedata/subject109.zip',sep = " ", names = col_names)

##Data analysis and preprocessing: subject 1

In [5]:
time101.head()

,ts,activityID,hr,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_5,IMU_hand_6,IMU_hand_7,...,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13,IMU_ankle_14,IMU_ankle_15,IMU_ankle_16,IMU_ankle_17
0,8.38,0,104.0,30.0,2.37223,8.60074,3.51048,2.43954,8.76165,3.35465,...,0.008300,0.009250,-0.017580,-61.1888,-38.9599,-58.1438,1.0,0.0,0.0,0.0
1,8.39,0,NaN,30.0,2.18837,8.56560,3.66179,2.39494,8.55081,3.64207,...,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253,1.0,0.0,0.0,0.0
2,8.40,0,NaN,30.0,2.37357,8.60107,3.54898,2.30514,8.53644,3.73280,...,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999,1.0,0.0,0.0,0.0
3,8.41,0,NaN,30.0,2.07473,8.52853,3.66021,2.33528,8.53622,3.73277,...,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956,1.0,0.0,0.0,0.0
4,8.42,0,NaN,30.0,2.22936,8.83122,3.70000,2.23055,8.59741,3.76295,...,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694,1.0,0.0,0.0,0.0


## Data Cleaning

### Dropping heart rate, 3D accelerometer data (scale 6g), since use of first accelerometer is recommended and orientation data which are invalid in this data collection. 

In [16]:
time101 = time101.drop(columns=['hr', 
                 'IMU_hand_5', 'IMU_hand_6', 'IMU_hand_7','IMU_hand_14', 'IMU_hand_15', 'IMU_hand_16','IMU_hand_17',
                 'IMU_chest_5', 'IMU_chest_6', 'IMU_chest_7','IMU_chest_15','IMU_chest_16', 'IMU_chest_17',
                'IMU_ankle_5', 'IMU_ankle_6','IMU_ankle_7','IMU_ankle_14','IMU_ankle_15', 'IMU_ankle_16', 'IMU_ankle_17'])

In [17]:
# checking column names
time101.columns

Index(['ts', 'activityID', 'IMU_hand_1', 'IMU_hand_2', 'IMU_hand_3',
       'IMU_hand_4', 'IMU_hand_8', 'IMU_hand_9', 'IMU_hand_10', 'IMU_hand_11',
       'IMU_hand_12', 'IMU_hand_13', 'IMU_chest_1', 'IMU_chest_2',
       'IMU_chest_3', 'IMU_chest_4', 'IMU_chest_8', 'IMU_chest_9',
       'IMU_chest_10', 'IMU_chest_11', 'IMU_chest_12', 'IMU_chest_13',
       'IMU_chest_14', 'IMU_ankle_1', 'IMU_ankle_2', 'IMU_ankle_3',
       'IMU_ankle_4', 'IMU_ankle_8', 'IMU_ankle_9', 'IMU_ankle_10',
       'IMU_ankle_11', 'IMU_ankle_12', 'IMU_ankle_13'],
      dtype='object')

In [18]:
# sorting values based on timestamp and axctivityID
time101 = time101.sort_values(["ts","activityID"], ascending = (True, True))

In [19]:
df_subect1 = time101.copy(deep=True)

In [20]:
df_subect1

,ts,activityID,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_8,IMU_hand_9,IMU_hand_10,IMU_hand_11,...,IMU_ankle_1,IMU_ankle_2,IMU_ankle_3,IMU_ankle_4,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13
0,8.38,0,30.0,2.37223,8.60074,3.51048,-0.092217,0.056812,-0.015845,14.6806,...,30.3125,9.65918,-1.655690,-0.099797,0.008300,0.009250,-0.017580,-61.1888,-38.9599,-58.1438
1,8.39,0,30.0,2.18837,8.56560,3.66179,-0.024413,0.047759,0.006474,14.8991,...,30.3125,9.69370,-1.579020,-0.215687,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253
2,8.40,0,30.0,2.37357,8.60107,3.54898,-0.057976,0.032574,-0.006988,14.2420,...,30.3125,9.58944,-1.732760,0.092914,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999
3,8.41,0,30.0,2.07473,8.52853,3.66021,-0.002352,0.032810,-0.003747,14.8908,...,30.3125,9.58814,-1.770400,0.054545,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956
4,8.42,0,30.0,2.22936,8.83122,3.70000,0.012269,0.018305,-0.053325,15.5612,...,30.3125,9.69771,-1.656250,-0.060809,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376412,3772.50,0,30.5,2.02477,7.29553,5.74194,1.516160,-0.044713,-0.085853,18.8018,...,33.3125,9.89247,-0.404788,-0.027988,0.048745,-0.008034,0.018600,-58.8381,-36.2397,-11.0980
376413,3772.51,0,30.5,2.10836,7.86504,5.85674,1.542230,-0.023898,-0.075649,17.6871,...,33.3125,9.77339,-0.440118,-0.260527,0.042113,0.024647,0.013375,-58.9537,-36.3379,-11.7170
376414,3772.52,0,30.5,2.07163,8.39581,5.77742,1.502390,-0.029592,-0.074682,18.0358,...,33.3125,9.81482,-0.517578,-0.105264,-0.014889,0.026009,0.025054,-59.1781,-35.8985,-10.9678
376415,3772.53,0,30.5,2.19569,8.77634,6.00892,1.332220,-0.052372,-0.034189,17.9474,...,33.3125,9.85194,-0.365619,-0.144416,0.047026,-0.035531,0.024640,-58.8299,-36.6845,-10.8553


In [21]:
# checking first and last value of each activity to check if timestamp overlaps
g = df_subect1.groupby('activityID')

(pd.concat([g.head(1), g.tail(1)])
   .drop_duplicates()
   .sort_values(['ts','activityID'])
   .reset_index(drop=True))

,ts,activityID,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_8,IMU_hand_9,IMU_hand_10,IMU_hand_11,...,IMU_ankle_1,IMU_ankle_2,IMU_ankle_3,IMU_ankle_4,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13
0,8.38,0,30.0000,2.372230,8.60074,3.51048,-0.092217,0.056812,-0.015845,14.680600,...,30.3125,9.65918,-1.655690,-0.099797,0.008300,0.009250,-0.017580,-61.1888,-38.959900,-58.14380
1,37.66,1,30.3750,2.215300,8.27915,5.58753,-0.004750,0.037579,-0.011145,8.932000,...,30.7500,9.73855,-1.847610,0.095156,0.002908,-0.027714,0.001752,-61.1081,-36.863600,-58.36960
2,309.52,1,31.9375,2.202600,9.39259,2.81249,0.072093,-0.016557,0.008165,14.894500,...,32.3750,9.77400,-0.315528,-1.573340,0.024080,-0.007107,0.022482,-83.6035,-37.861900,-56.33960
3,309.53,2,31.9375,2.124620,9.24108,2.77335,0.037944,0.024816,-0.016013,15.003300,...,32.3750,9.77730,-0.316492,-1.457530,-0.017022,0.013759,0.013117,-83.3840,-37.322900,-55.96430
4,544.32,2,32.6875,2.476500,9.01417,3.16426,0.079567,-0.032023,0.018626,-15.477400,...,33.0000,9.80320,-0.209974,-0.570367,0.011498,0.002644,-0.016846,-85.7207,37.235400,-5.88598
5,544.33,3,32.6875,2.552340,9.24089,3.12614,0.109104,0.012499,0.007080,-15.694900,...,33.0625,9.80611,-0.286875,-0.454091,0.016202,0.071309,-0.037459,-86.2875,37.696600,-5.87642
6,761.49,3,33.1250,2.226530,8.35771,5.89533,0.184558,0.033761,0.002067,17.425100,...,33.5625,9.76257,-1.881960,-0.367459,-0.004201,-0.065393,0.024225,-44.1271,37.687400,-59.69380
7,858.05,17,33.2500,2.438590,7.02674,5.74905,-0.075247,0.070647,-0.040244,-45.900900,...,33.7500,9.85126,-0.707658,-0.103710,0.023601,-0.018856,-0.011502,-59.9952,-13.791600,38.53860
8,1093.77,17,33.3750,2.204660,9.36501,3.96724,-0.749754,0.103855,-0.012051,-48.772900,...,34.1875,9.62987,-1.571080,-1.142540,-0.014103,0.077143,0.022384,-58.6256,-3.455900,45.94060
9,1195.98,16,33.5000,1.996200,9.24795,3.42568,0.198887,0.023091,0.000238,-26.806300,...,34.3750,9.72493,-1.501400,-0.447386,-0.010609,-0.000836,0.035841,-30.9601,26.283800,-16.35300


In [22]:
# checking data types
df_subect1.dtypes

ts              float64
activityID        int64
IMU_hand_1      float64
IMU_hand_2      float64
IMU_hand_3      float64
IMU_hand_4      float64
IMU_hand_8      float64
IMU_hand_9      float64
IMU_hand_10     float64
IMU_hand_11     float64
IMU_hand_12     float64
IMU_hand_13     float64
IMU_chest_1     float64
IMU_chest_2     float64
IMU_chest_3     float64
IMU_chest_4     float64
IMU_chest_8     float64
IMU_chest_9     float64
IMU_chest_10    float64
IMU_chest_11    float64
IMU_chest_12    float64
IMU_chest_13    float64
IMU_chest_14    float64
IMU_ankle_1     float64
IMU_ankle_2     float64
IMU_ankle_3     float64
IMU_ankle_4     float64
IMU_ankle_8     float64
IMU_ankle_9     float64
IMU_ankle_10    float64
IMU_ankle_11    float64
IMU_ankle_12    float64
IMU_ankle_13    float64
dtype: object

In [23]:
# converting float64 to datetime
df_subect1['ts'] = pd.to_datetime(df_subect1['ts'], unit='s')
df_subect1.head()

,ts,activityID,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_8,IMU_hand_9,IMU_hand_10,IMU_hand_11,...,IMU_ankle_1,IMU_ankle_2,IMU_ankle_3,IMU_ankle_4,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13
0,1970-01-01 00:00:08.380,0,30.0,2.37223,8.60074,3.51048,-0.092217,0.056812,-0.015845,14.6806,...,30.3125,9.65918,-1.65569,-0.099797,0.008300,0.009250,-0.017580,-61.1888,-38.9599,-58.1438
1,1970-01-01 00:00:08.390,0,30.0,2.18837,8.56560,3.66179,-0.024413,0.047759,0.006474,14.8991,...,30.3125,9.69370,-1.57902,-0.215687,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253
2,1970-01-01 00:00:08.400,0,30.0,2.37357,8.60107,3.54898,-0.057976,0.032574,-0.006988,14.2420,...,30.3125,9.58944,-1.73276,0.092914,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999
3,1970-01-01 00:00:08.410,0,30.0,2.07473,8.52853,3.66021,-0.002352,0.032810,-0.003747,14.8908,...,30.3125,9.58814,-1.77040,0.054545,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956
4,1970-01-01 00:00:08.420,0,30.0,2.22936,8.83122,3.70000,0.012269,0.018305,-0.053325,15.5612,...,30.3125,9.69771,-1.65625,-0.060809,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694


In [24]:
#set timestamp as index
df_subect1 = df_subect1.set_index('ts')
df_subect1.head()

,activityID,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_8,IMU_hand_9,IMU_hand_10,IMU_hand_11,IMU_hand_12,...,IMU_ankle_1,IMU_ankle_2,IMU_ankle_3,IMU_ankle_4,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13
ts,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:08.380,0,30.0,2.37223,8.60074,3.51048,-0.092217,0.056812,-0.015845,14.6806,-69.2128,...,30.3125,9.65918,-1.65569,-0.099797,0.008300,0.009250,-0.017580,-61.1888,-38.9599,-58.1438
1970-01-01 00:00:08.390,0,30.0,2.18837,8.56560,3.66179,-0.024413,0.047759,0.006474,14.8991,-69.2224,...,30.3125,9.69370,-1.57902,-0.215687,-0.006577,-0.004638,0.000368,-59.8479,-38.8919,-58.5253
1970-01-01 00:00:08.400,0,30.0,2.37357,8.60107,3.54898,-0.057976,0.032574,-0.006988,14.2420,-69.5197,...,30.3125,9.58944,-1.73276,0.092914,0.003014,0.000148,0.022495,-60.7361,-39.4138,-58.3999
1970-01-01 00:00:08.410,0,30.0,2.07473,8.52853,3.66021,-0.002352,0.032810,-0.003747,14.8908,-69.5439,...,30.3125,9.58814,-1.77040,0.054545,0.003175,-0.020301,0.011275,-60.4091,-38.7635,-58.3956
1970-01-01 00:00:08.420,0,30.0,2.22936,8.83122,3.70000,0.012269,0.018305,-0.053325,15.5612,-68.8196,...,30.3125,9.69771,-1.65625,-0.060809,0.012698,-0.014303,-0.002823,-61.5199,-39.3879,-58.2694


In [25]:
#excluding activityID to avoid data corruption (it creates extra NaNs)

df_subect1_gr = df_subect1.groupby(['activityID', pd.Grouper(freq='33L')])[['IMU_hand_1', 'IMU_hand_2', 'IMU_hand_3', 'IMU_hand_4',
       'IMU_hand_8', 'IMU_hand_9', 'IMU_hand_10', 'IMU_hand_11', 'IMU_hand_12',
       'IMU_hand_13', 'IMU_chest_1', 'IMU_chest_2', 'IMU_chest_3',
       'IMU_chest_4', 'IMU_chest_8', 'IMU_chest_9', 'IMU_chest_10',
       'IMU_chest_11', 'IMU_chest_12', 'IMU_chest_13', 'IMU_chest_14',
       'IMU_ankle_1', 'IMU_ankle_2', 'IMU_ankle_3', 'IMU_ankle_4',
       'IMU_ankle_8', 'IMU_ankle_9', 'IMU_ankle_10', 'IMU_ankle_11',
       'IMU_ankle_12', 'IMU_ankle_13']].mean()

In [26]:
# checking data size after downsampling
print("shape before downsampling ",df_subect1.shape)
print("shape after downsampling ",df_subect1_gr.shape)

shape before downsampling  (376417, 32)
shape after downsampling  (114087, 31)


In [27]:
df_subect1_gr.head()

IMU_hand_1  IMU_hand_2  IMU_hand_3  \
activityID ts                                                            
0          1970-01-01 00:00:08.349        30.0    2.372230    8.600740   
           1970-01-01 00:00:08.382        30.0    2.212223    8.565067   
           1970-01-01 00:00:08.415        30.0    2.288777    8.829837   
           1970-01-01 00:00:08.448        30.0    2.317010    9.046540   
           1970-01-01 00:00:08.481        30.0    2.408680    8.789473   

                                    IMU_hand_4  IMU_hand_8  IMU_hand_9  \
activityID ts                                                            
0          1970-01-01 00:00:08.349    3.510480   -0.092217    0.056812   
           1970-01-01 00:00:08.382    3.623660   -0.028247    0.037714   
           1970-01-01 00:00:08.415    3.598257   -0.002732   -0.000318   
           1970-01-01 00:00:08.448    3.460035    0.009041   -0.049678   
           1970-01-01 00:00:08.481    3.433410   -0.000315   -0.051990   

                                    IMU_hand_10  IMU_hand_11  IMU_hand_12  \
activityID ts                                                               
0          1970-01-01 00:00:08.349    -0.015845    14.680600   -69.212800   
           1970-01-01 00:00:08.382    -0.001420    14.677300   -69.428667   
           1970-01-01 00:00:08.415    -0.051894    15.379433   -68.739000   
           1970-01-01 00:00:08.448    -0.022555    14.875875   -68.951300   
           1970-01-01 00:00:08.481    -0.027070    15.209967   -68.812633   

                                    IMU_hand_13  ...  IMU_ankle_1  \
activityID ts                                    ...                
0          1970-01-01 00:00:08.349    -5.589050  ...      30.3125   
           1970-01-01 00:00:08.382    -5.707067  ...      30.3125   
           1970-01-01 00:00:08.415    -6.093440  ...      30.3125   
           1970-01-01 00:00:08.448    -5.646330  ...      30.3125   
           1970-01-01 00:00:08.481    -5.243613  ...      30.3125   

                                    IMU_ankle_2  IMU_ankle_3  IMU_ankle_4  \
activityID ts                                                               
0          1970-01-01 00:00:08.349     9.659180    -1.655690    -0.099797   
           1970-01-01 00:00:08.382     9.623760    -1.694060    -0.022743   
           1970-01-01 00:00:08.415     9.709320    -1.681423    -0.086260   
           1970-01-01 00:00:08.448     9.738735    -1.600493     0.055035   
           1970-01-01 00:00:08.481     9.685600    -1.656277    -0.048069   

                                    IMU_ankle_8  IMU_ankle_9  IMU_ankle_10  \
activityID ts                                                                
0          1970-01-01 00:00:08.349     0.008300     0.009250     -0.017580   
           1970-01-01 00:00:08.382    -0.000129    -0.008263      0.011380   
           1970-01-01 00:00:08.415    -0.008455    -0.028087      0.004607   
           1970-01-01 00:00:08.448     0.020163    -0.015405      0.012941   
           1970-01-01 00:00:08.481     0.037468    -0.012702      0.001459   

                                    IMU_ankle_11  IMU_ankle_12  IMU_ankle_13  
activityID ts                                                                 
0          1970-01-01 00:00:08.349    -61.188800    -38.959900    -58.143800  
           1970-01-01 00:00:08.382    -60.331033    -39.023067    -58.440267  
           1970-01-01 00:00:08.415    -60.815333    -39.044433    -58.312733  
           1970-01-01 00:00:08.448    -61.047450    -39.101550    -58.239250  
           1970-01-01 00:00:08.481    -60.631433    -38.794600    -58.229067  

[5 rows x 31 columns]

In [22]:
# df_subect1_gr.index.levels[0]

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24], dtype='int64', name='activityID')

In [28]:
#reset index to keep just one index, ts.
df_subect1 = df_subect1_gr.reset_index(level='activityID')
df_subect1.head()

,activityID,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_8,IMU_hand_9,IMU_hand_10,IMU_hand_11,IMU_hand_12,...,IMU_ankle_1,IMU_ankle_2,IMU_ankle_3,IMU_ankle_4,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13
ts,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:08.349,0,30.0,2.372230,8.600740,3.510480,-0.092217,0.056812,-0.015845,14.680600,-69.212800,...,30.3125,9.659180,-1.655690,-0.099797,0.008300,0.009250,-0.017580,-61.188800,-38.959900,-58.143800
1970-01-01 00:00:08.382,0,30.0,2.212223,8.565067,3.623660,-0.028247,0.037714,-0.001420,14.677300,-69.428667,...,30.3125,9.623760,-1.694060,-0.022743,-0.000129,-0.008263,0.011380,-60.331033,-39.023067,-58.440267
1970-01-01 00:00:08.415,0,30.0,2.288777,8.829837,3.598257,-0.002732,-0.000318,-0.051894,15.379433,-68.739000,...,30.3125,9.709320,-1.681423,-0.086260,-0.008455,-0.028087,0.004607,-60.815333,-39.044433,-58.312733
1970-01-01 00:00:08.448,0,30.0,2.317010,9.046540,3.460035,0.009041,-0.049678,-0.022555,14.875875,-68.951300,...,30.3125,9.738735,-1.600493,0.055035,0.020163,-0.015405,0.012941,-61.047450,-39.101550,-58.239250
1970-01-01 00:00:08.481,0,30.0,2.408680,8.789473,3.433410,-0.000315,-0.051990,-0.027070,15.209967,-68.812633,...,30.3125,9.685600,-1.656277,-0.048069,0.037468,-0.012702,0.001459,-60.631433,-38.794600,-58.229067


## Missing Values

In [29]:
# checking NaN statistics
df_subect1.isna().sum()

activityID       0
IMU_hand_1      39
IMU_hand_2      39
IMU_hand_3      39
IMU_hand_4      39
IMU_hand_8      39
IMU_hand_9      39
IMU_hand_10     39
IMU_hand_11     39
IMU_hand_12     39
IMU_hand_13     39
IMU_chest_1     45
IMU_chest_2     45
IMU_chest_3     45
IMU_chest_4     45
IMU_chest_8     45
IMU_chest_9     45
IMU_chest_10    45
IMU_chest_11    45
IMU_chest_12    45
IMU_chest_13    45
IMU_chest_14    45
IMU_ankle_1     56
IMU_ankle_2     56
IMU_ankle_3     56
IMU_ankle_4     56
IMU_ankle_8     56
IMU_ankle_9     56
IMU_ankle_10    56
IMU_ankle_11    56
IMU_ankle_12    56
IMU_ankle_13    56
dtype: int64

###Filling missing values with linear interpolation

In [30]:
# df_sub1_data = df_sub1_data.set_index('ts')
df_sub1_data = df_subect1.interpolate()

In [31]:
#checking NaNs again 
df_sub1_data.isna().sum()

activityID      0
IMU_hand_1      0
IMU_hand_2      0
IMU_hand_3      0
IMU_hand_4      0
IMU_hand_8      0
IMU_hand_9      0
IMU_hand_10     0
IMU_hand_11     0
IMU_hand_12     0
IMU_hand_13     0
IMU_chest_1     0
IMU_chest_2     0
IMU_chest_3     0
IMU_chest_4     0
IMU_chest_8     0
IMU_chest_9     0
IMU_chest_10    0
IMU_chest_11    0
IMU_chest_12    0
IMU_chest_13    0
IMU_chest_14    0
IMU_ankle_1     0
IMU_ankle_2     0
IMU_ankle_3     0
IMU_ankle_4     0
IMU_ankle_8     0
IMU_ankle_9     0
IMU_ankle_10    0
IMU_ankle_11    0
IMU_ankle_12    0
IMU_ankle_13    0
dtype: int64

In [32]:
df_sub1_data.head()

,activityID,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_8,IMU_hand_9,IMU_hand_10,IMU_hand_11,IMU_hand_12,...,IMU_ankle_1,IMU_ankle_2,IMU_ankle_3,IMU_ankle_4,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13
ts,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:08.349,0,30.0,2.372230,8.600740,3.510480,-0.092217,0.056812,-0.015845,14.680600,-69.212800,...,30.3125,9.659180,-1.655690,-0.099797,0.008300,0.009250,-0.017580,-61.188800,-38.959900,-58.143800
1970-01-01 00:00:08.382,0,30.0,2.212223,8.565067,3.623660,-0.028247,0.037714,-0.001420,14.677300,-69.428667,...,30.3125,9.623760,-1.694060,-0.022743,-0.000129,-0.008263,0.011380,-60.331033,-39.023067,-58.440267
1970-01-01 00:00:08.415,0,30.0,2.288777,8.829837,3.598257,-0.002732,-0.000318,-0.051894,15.379433,-68.739000,...,30.3125,9.709320,-1.681423,-0.086260,-0.008455,-0.028087,0.004607,-60.815333,-39.044433,-58.312733
1970-01-01 00:00:08.448,0,30.0,2.317010,9.046540,3.460035,0.009041,-0.049678,-0.022555,14.875875,-68.951300,...,30.3125,9.738735,-1.600493,0.055035,0.020163,-0.015405,0.012941,-61.047450,-39.101550,-58.239250
1970-01-01 00:00:08.481,0,30.0,2.408680,8.789473,3.433410,-0.000315,-0.051990,-0.027070,15.209967,-68.812633,...,30.3125,9.685600,-1.656277,-0.048069,0.037468,-0.012702,0.001459,-60.631433,-38.794600,-58.229067


## Data Segmentation

In [125]:
# Conver dataframe to numpy array
np_df_sub1 = df_sub1_data.to_numpy()

In [34]:
np_df_sub1

array([[  0.        ,  30.        ,   2.37223   , ..., -61.1888    ,
        -38.9599    , -58.1438    ],
       [  0.        ,  30.        ,   2.21222333, ..., -60.33103333,
        -39.02306667, -58.44026667],
       [  0.        ,  30.        ,   2.28877667, ..., -60.81533333,
        -39.04443333, -58.31273333],
       ...,
       [ 24.        ,  30.1875    ,   2.40800667, ..., -56.33313333,
        -29.4948    ,  34.2977    ],
       [ 24.        ,  30.1875    ,   2.56870333, ..., -55.91616667,
        -29.62683333,  34.82923333],
       [ 24.        ,  30.1875    ,   2.51044   , ..., -56.3426    ,
        -29.1916    ,  33.6409    ]])

Segmentation of IMU time series data into 2 second windows (60 rows) with .5 second overlap (45 rows)

In [126]:
from numpy.lib.stride_tricks import sliding_window_view
import numpy as np
values = np_df_sub1
window_data_sub1 = sliding_window_view(values, window_shape = (60,32))[::45]

In [127]:
window_data_sub1.shape

(2534, 1, 60, 32)

### convert numpy array to dataframe again

In [128]:
columns = ['activityID','IMU_hand_1', 'IMU_hand_2', 'IMU_hand_3', 'IMU_hand_4',
       'IMU_hand_8', 'IMU_hand_9', 'IMU_hand_10', 'IMU_hand_11', 'IMU_hand_12',
       'IMU_hand_13', 'IMU_chest_1', 'IMU_chest_2', 'IMU_chest_3',
       'IMU_chest_4', 'IMU_chest_8', 'IMU_chest_9', 'IMU_chest_10',
       'IMU_chest_11', 'IMU_chest_12', 'IMU_chest_13', 'IMU_chest_14',
       'IMU_ankle_1', 'IMU_ankle_2', 'IMU_ankle_3', 'IMU_ankle_4',
       'IMU_ankle_8', 'IMU_ankle_9', 'IMU_ankle_10', 'IMU_ankle_11',
       'IMU_ankle_12', 'IMU_ankle_13',"window_no"]

In [129]:
# this block of code is converting the np array to a dataframe. Each numpy array is mapped to a row in df which represents 2 second window. 
# at the end of each 60 row extraction, it adds a new column assigning the window number
df_windowed = pd.DataFrame()
for i in range(len(window_data_sub1)):
  df_sample = pd.DataFrame()
  for j in range(len(window_data_sub1[0][0])):
    df_sample = df_sample.append(pd.DataFrame(window_data_sub1[i][0][j].reshape(-1, len(window_data_sub1[i][0][j]))),ignore_index=True)
    df_sample['window_no']= f'window{i}'
  df_windowed = df_windowed.append(df_sample)

In [130]:
# reassign the column names
df_windowed.columns = columns
df_windowed.head()

,activityID,IMU_hand_1,IMU_hand_2,IMU_hand_3,IMU_hand_4,IMU_hand_8,IMU_hand_9,IMU_hand_10,IMU_hand_11,IMU_hand_12,...,IMU_ankle_2,IMU_ankle_3,IMU_ankle_4,IMU_ankle_8,IMU_ankle_9,IMU_ankle_10,IMU_ankle_11,IMU_ankle_12,IMU_ankle_13,window_no
0,0.0,30.0,2.372230,8.600740,3.510480,-0.092217,0.056812,-0.015845,14.680600,-69.212800,...,9.659180,-1.655690,-0.099797,0.008300,0.009250,-0.017580,-61.188800,-38.959900,-58.143800,window0
1,0.0,30.0,2.212223,8.565067,3.623660,-0.028247,0.037714,-0.001420,14.677300,-69.428667,...,9.623760,-1.694060,-0.022743,-0.000129,-0.008263,0.011380,-60.331033,-39.023067,-58.440267,window0
2,0.0,30.0,2.288777,8.829837,3.598257,-0.002732,-0.000318,-0.051894,15.379433,-68.739000,...,9.709320,-1.681423,-0.086260,-0.008455,-0.028087,0.004607,-60.815333,-39.044433,-58.312733,window0
3,0.0,30.0,2.317010,9.046540,3.460035,0.009041,-0.049678,-0.022555,14.875875,-68.951300,...,9.738735,-1.600493,0.055035,0.020163,-0.015405,0.012941,-61.047450,-39.101550,-58.239250,window0
4,0.0,30.0,2.408680,8.789473,3.433410,-0.000315,-0.051990,-0.027070,15.209967,-68.812633,...,9.685600,-1.656277,-0.048069,0.037468,-0.012702,0.001459,-60.631433,-38.794600,-58.229067,window0


In [45]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


## Features Extraction:

In [131]:
data_1 = df_windowed

In [ ]:
data_1.isna().sum()

In [133]:
# get features based on wach window
data_1 = data_1.groupby(['activityID','window_no']).agg(['min','max','mean'])

In [134]:
data_1 = data_1.reset_index(level='activityID')

In [135]:
data_1 = data_1.reset_index(drop=True)

In [136]:
data_1.isna().sum().sum()

0

In [ ]:
data_1

In [138]:
#saving csv file to drive
data_1.to_csv('Featured_subject101.csv')
!cp Featured_subject101.csv "drive/My Drive/timedata"

In [144]:
# a sample of the numopy array 
window_data_sub1[110][0][59]

array([ 0.00000000e+00,  3.34375000e+01,  4.93215667e+00,  7.11650000e+00,
        1.02648900e+00, -9.15117333e-01, -2.43472000e-01,  1.09260800e+00,
       -4.18748000e+01, -3.95936333e+01, -3.07905000e+00,  3.53750000e+01,
       -8.98864333e-01,  9.46726667e+00, -3.17988000e+00,  7.75263667e-03,
       -6.67481333e-01,  3.65386333e-02, -6.52001667e+00, -5.19773333e+01,
       -4.82678667e+00,  1.00000000e+00,  3.42500000e+01,  8.71463000e+00,
        6.56210333e-01, -4.19684200e-01,  6.84116667e-01, -2.64355000e-01,
        5.75178667e-02, -5.28704000e+01, -7.06062333e+00,  4.45347667e+01])

In [139]:
df_segmented_data_all = df_windowed
df_featured_data_all = data_1
df_featured_data_all["userID"] = 101

In [141]:
df_featured_data_all

activityID IMU_hand_1                     IMU_hand_2                      \
                       min      max       mean        min       max      mean   
0           0.0    30.0000  30.0000  30.000000   1.997705  3.612930  2.543853   
1           0.0    30.0000  30.0625  30.044097   1.997705  3.265913  2.718658   
2           0.0    30.1875  30.2500  30.234028   2.033703  2.423837  2.277450   
3           0.0    33.4375  33.4375  33.437500  -7.156047 -1.548803 -4.336507   
4           0.0    33.4375  33.4375  33.437500  -5.040770 -1.207553 -3.654563   
...         ...        ...      ...        ...        ...       ...       ...   
2546       24.0    30.1875  30.2500  30.237240 -32.584667  6.422820 -6.267235   
2547       24.0    30.1875  30.2500  30.213281 -31.456350  7.347313 -5.515224   
2548       24.0    30.2500  30.2500  30.250000 -13.138233  0.839542 -4.227147   
2549       24.0    30.2500  30.2500  30.250000  -4.775343  6.391227  1.248520   
2550       24.0    30.1875  30.2500  30.231597   1.777540  6.391227  3.181651   

     IMU_hand_3                       ... IMU_ankle_11                        \
            min        max      mean  ...          min        max       mean   
0      7.468330   9.822703  8.916264  ...   -61.709833 -60.331033 -60.899287   
1      7.468330   9.362150  8.674966  ...   -61.413600 -60.557467 -60.968967   
2      8.491300   9.117313  8.728172  ...   -61.516625 -60.495067 -60.972977   
3      5.505943  11.062267  8.175816  ...   -99.743367 -85.678175 -92.921164   
4      7.858473   9.709997  8.947013  ...   -87.286800 -85.426533 -86.003325   
...         ...        ...       ...  ...          ...        ...        ...   
2546  -0.679849  18.104600  7.998846  ...   -55.686333 -39.684033 -49.046881   
2547  -0.274725  19.833567  8.671105  ...   -57.157567 -37.631200 -49.903575   
2548   1.476146  15.147233  8.521648  ...   -58.828433 -46.590450 -56.610587   
2549   4.277817  12.663167  7.854603  ...   -58.520533 -55.483675 -57.301192   
2550   4.818595  11.530833  8.108351  ...   -57.366067 -55.483675 -56.530972   

     IMU_ankle_12                       IMU_ankle_13                        \
              min        max       mean          min        max       mean   
0      -39.281067 -37.912933 -38.769602   -58.470200 -57.448900 -57.964617   
1      -39.519167 -37.912933 -38.652070   -58.468967 -57.372975 -57.964684   
2      -38.513467 -37.124767 -37.939436   -58.636500 -57.558200 -58.067073   
3       -8.937080   5.556420  -1.486670    31.679475  48.902333  43.404625   
4       -0.200197   5.778607   4.583928    45.838400  49.839900  48.608067   
...           ...        ...        ...          ...        ...        ...   
2546   -37.450200 -24.563000 -32.442600    34.843900  47.239967  40.965072   
2547   -41.483900 -22.046300 -31.954863    35.487433  45.962467  40.161373   
2548   -32.411333 -20.355533 -28.136719    28.148725  44.098467  33.935556   
2549   -32.411333 -29.342925 -31.199983    28.146833  34.995450  30.705578   
2550   -30.561033 -27.495667 -28.560712    32.457025  35.230167  34.527785   

     userID  
             
0       101  
1       101  
2       101  
3       101  
4       101  
...     ...  
2546    101  
2547    101  
2548    101  
2549    101  
2550    101  

[2551 rows x 95 columns]

# repeat procedure for the other 8 subjects

In [142]:
def data_loader(df):
    """mean of accelerometer data"""
    col_names = ['ts','activityID','hr']+[f'IMU_hand_{x+1}' for x in range(17)]+[f'IMU_chest_{x+1}' for x in range(17)]+[f'IMU_ankle_{x+1}' for x in range(17)]
    # df should be sent dynamically
    df = df.drop(columns=['hr', 
                    'IMU_hand_5', 'IMU_hand_6', 'IMU_hand_7','IMU_hand_14', 'IMU_hand_15', 'IMU_hand_16','IMU_hand_17',
                    'IMU_chest_5', 'IMU_chest_6', 'IMU_chest_7','IMU_chest_15','IMU_chest_16', 'IMU_chest_17',
                    'IMU_ankle_5', 'IMU_ankle_6','IMU_ankle_7','IMU_ankle_14','IMU_ankle_15', 'IMU_ankle_16', 'IMU_ankle_17'])
    df = df.sort_values(["ts","activityID"], ascending = (True, True))
    df['ts'] = pd.to_datetime(df['ts'], unit='s')
    df = df.set_index('ts')
    return df
def downsampler(df):
  #excluding activityID to avoid data corruption (it creates extra NaNs)
  df_temp = df.groupby(['activityID', pd.Grouper(freq='33L')])[['IMU_hand_1', 'IMU_hand_2', 'IMU_hand_3', 'IMU_hand_4',
        'IMU_hand_8', 'IMU_hand_9', 'IMU_hand_10', 'IMU_hand_11', 'IMU_hand_12',
        'IMU_hand_13', 'IMU_chest_1', 'IMU_chest_2', 'IMU_chest_3',
        'IMU_chest_4', 'IMU_chest_8', 'IMU_chest_9', 'IMU_chest_10',
        'IMU_chest_11', 'IMU_chest_12', 'IMU_chest_13', 'IMU_chest_14',
        'IMU_ankle_1', 'IMU_ankle_2', 'IMU_ankle_3', 'IMU_ankle_4',
        'IMU_ankle_8', 'IMU_ankle_9', 'IMU_ankle_10', 'IMU_ankle_11',
        'IMU_ankle_12', 'IMU_ankle_13']].mean()
      # checking data size after downsampling
  print("shape before downsampling ",df.shape)
  print("shape after downsampling ",df_temp.shape)
  #reset index to keep just one index, ts.
  df_temp = df_temp.reset_index(level='activityID')
  return df_temp

def clean_interpolate(df):
  # checking NaN statistics
  print("NaNs before interpolating",df.isna().sum())
  df = df.interpolate()
  print("NaNs after interpolating",df.isna().sum())
  return df

def segmentationWindow(df):
  # data segmentation
  np_df_sub1 = df.to_numpy()
  values = np_df_sub1
  window_data_sub1 = sliding_window_view(values, window_shape = (60,32))[::45]
  print("data shape", window_data_sub1.shape)

  columns = ['activityID','IMU_hand_1', 'IMU_hand_2', 'IMU_hand_3', 'IMU_hand_4',
       'IMU_hand_8', 'IMU_hand_9', 'IMU_hand_10', 'IMU_hand_11', 'IMU_hand_12',
       'IMU_hand_13', 'IMU_chest_1', 'IMU_chest_2', 'IMU_chest_3',
       'IMU_chest_4', 'IMU_chest_8', 'IMU_chest_9', 'IMU_chest_10',
       'IMU_chest_11', 'IMU_chest_12', 'IMU_chest_13', 'IMU_chest_14',
       'IMU_ankle_1', 'IMU_ankle_2', 'IMU_ankle_3', 'IMU_ankle_4',
       'IMU_ankle_8', 'IMU_ankle_9', 'IMU_ankle_10', 'IMU_ankle_11',
       'IMU_ankle_12', 'IMU_ankle_13','window_no']
  # this block of code is converting the np array to a dataframe. Each numpy array is mapped to a row in df which represents 2 second window. 
  # at the end of each 60 row extraction, it adds a new column assigning the window number
  df_windowed = pd.DataFrame()
  for i in range(len(window_data_sub1)):
    df_sample = pd.DataFrame()
    for j in range(len(window_data_sub1[0][0])):
      df_sample = df_sample.append(pd.DataFrame(window_data_sub1[i][0][j].reshape(-1, len(window_data_sub1[i][0][j]))),ignore_index=True)
      df_sample['window_no']= f'window{i}'
    df_windowed = df_windowed.append(df_sample)
  # reassign the column names
  df_windowed.columns = columns
  return df_windowed

def FeatureExtraction(df):
  # feature extraction
  data_1 = df # df coming from segmentation window
  data_1 = data_1.groupby(['activityID','window_no']).agg(['min','max','mean'])
  data_1 = data_1.reset_index(level='activityID')
  data_1 = data_1.reset_index(drop=True)
  return data_1

In [ ]:
#saving csv file to drive
# data_1.to_csv('Featured_subject101.csv')
# !cp Featured_subject101.csv "drive/My Drive/timedata"

In [143]:
list_subject = [102, 103, 104,105,104,106, 107,108, 109]

for item in list_subject:
  data = data_loader(time102)
  sampled_data = downsampler(data)
  cleaned_data = clean_interpolate(sampled_data)
  windowed = segmentationWindow(cleaned_data) #save the output to combined windowed df
  win_df = windowed
  df_segmented_data_all = df_segmented_data_all.append(win_df)
  featured = FeatureExtraction(windowed) #save the output to combined feature extracted df
  featured["userID"] = item
  df_featured_data_all = df_featured_data_all.append(featured)

shape before downsampling  (447000, 32)
shape after downsampling  (135477, 31)
NaNs before interpolating activityID        0
IMU_hand_1       99
IMU_hand_2       99
IMU_hand_3       99
IMU_hand_4       99
IMU_hand_8       99
IMU_hand_9       99
IMU_hand_10      99
IMU_hand_11      99
IMU_hand_12      99
IMU_hand_13      99
IMU_chest_1      45
IMU_chest_2      45
IMU_chest_3      45
IMU_chest_4      45
IMU_chest_8      45
IMU_chest_9      45
IMU_chest_10     45
IMU_chest_11     45
IMU_chest_12     45
IMU_chest_13     45
IMU_chest_14     45
IMU_ankle_1     144
IMU_ankle_2     144
IMU_ankle_3     144
IMU_ankle_4     144
IMU_ankle_8     144
IMU_ankle_9     144
IMU_ankle_10    144
IMU_ankle_11    144
IMU_ankle_12    144
IMU_ankle_13    144
dtype: int64
NaNs after interpolating activityID      0
IMU_hand_1      0
IMU_hand_2      0
IMU_hand_3      0
IMU_hand_4      0
IMU_hand_8      0
IMU_hand_9      0
IMU_hand_10     0
IMU_hand_11     0
IMU_hand_12     0
IMU_hand_13     0
IMU_chest_1     0
I

In [144]:
df_segmented_data_all.isna().sum().sum()
df_segmented_data_all.shape

(1777440, 33)

In [147]:
df_segmented_data_all.to_csv('all_user_window_data_v2.csv')
!cp all_user_window_data_v2.csv "drive/My Drive/timedata"

In [145]:
df_featured_data_all.isna().sum().sum()

0

In [149]:
df_featured_data_all.to_csv('all_user_window_data_with_feature.csv')
!cp all_user_window_data_with_feature.csv "drive/My Drive/timedata"

## combine data

In [ ]:
# save all the windowed data to a df:
#save all the feature extracted data to a df:

# 5 fold cross validation using Random Forest (user dependent)



In [150]:
dataset = df_featured_data_all
print(dataset.dtypes)

activityID            float64
IMU_hand_1    min     float64
              max     float64
              mean    float64
IMU_hand_2    min     float64
                       ...   
IMU_ankle_12  mean    float64
IMU_ankle_13  min     float64
              max     float64
              mean    float64
userID                  int64
Length: 95, dtype: object


In [180]:
# dataset.head()
dataset.iloc[:, 0]

# dataset.iloc[:,1:95]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
3021    24.0
3022    24.0
3023    24.0
3024    24.0
3025    24.0
Name: (activityID, ), Length: 29785, dtype: float64

array([[ 30.        ,  30.        ,  30.        , ..., -57.4489    ,
        -57.96461681, 101.        ],
       [ 30.        ,  30.0625    ,  30.04409722, ..., -57.372975  ,
        -57.96468431, 101.        ],
       [ 30.1875    ,  30.25      ,  30.23402778, ..., -57.5582    ,
        -58.0670725 , 101.        ],
       ...,
       [ 28.5       ,  28.5625    ,  28.53611111, ...,  36.9895    ,
         36.00612361, 109.        ],
       [ 28.5       ,  28.5625    ,  28.54409722, ...,  36.86636667,
         36.23479889, 109.        ],
       [ 28.5       ,  28.5625    ,  28.55173611, ...,  36.62183333,
         36.11869611, 109.        ]])

In [181]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score


X, y = dataset.iloc[:,1:95].to_numpy(), dataset.iloc[:, 0].to_numpy()

X.shape, y.shape

z = dataset["userID"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=z)

clf = RandomForestClassifier(random_state=4,n_estimators = 500)

scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')

In [185]:
from sklearn.metrics import f1_score

scores = []
for i in range(5):
  X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=z)
  clf = RandomForestClassifier(random_state=4,n_estimators = 500)
  clf.fit(X, y)
  y_predicted = clf.predict(X_test)
  scores.append(f1_score(y_test,y_predicted,average = 'macro'))

# print("F1 micro: %1.4f\n" % f1_score(y_test, y_predicted, average='micro'))

In [186]:
scores

[1.0, 1.0, 1.0, 1.0, 1.0]

# 5 fold cross validation (User independent)

In [2]:
df_featured_data_all = pd.read_csv('/content/drive/MyDrive/timedata/all_user_window_data_with_feature.csv',index_col=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_featured_data_all.drop(index=df_featured_data_all.index[0], axis=0, inplace=True)

In [4]:
df_featured_data_all = df_featured_data_all.iloc[: , 1:]

In [5]:
dataset2 = df_featured_data_all.copy(deep = True)


In [7]:
data_check = dataset2.copy(deep = True)

In [17]:
test_data = data_check.loc[(data_check['userID'] == 103) | (data_check['userID'] == 105)|(data_check['userID'] == 108)]

train_data = data_check.loc[(data_check['userID'] != 103) & (data_check['userID'] != 105)&(data_check['userID'] != 108)]

In [27]:
y = dataset2.iloc[:, 0].to_numpy()
X = dataset2.iloc[:,1:95].to_numpy()

In [19]:
X_train, X_test, y_train, y_test = train_data.iloc[:,1:95].to_numpy(), test_data.iloc[:,1:95].to_numpy(), train_data.iloc[:, 0].to_numpy(), test_data.iloc[:, 0].to_numpy()

In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

In [28]:
X.shape

(29785, 94)

In [29]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=4,n_estimators = 500)

scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')

In [30]:
scores

array([0.74184558, 0.99913065, 1.        , 1.        , 1.        ])

# Deep learning model validation (User independent)

## Multilayer Perceptron

In [ ]:
from keras import metrics
model.compile(loss='binary_crossentropy', optimizer='adam',
metrics=[metrics.categorical_accuracy])

In [ ]:
# first neural network with keras tutorial
from numpy import loadtxt
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# define the keras model
y = dataset2.iloc[:, 0].to_numpy().astype(np.float32)
X = dataset2.iloc[:,1:95].to_numpy().astype(np.float32)
model = Sequential()
model.add(Dense(12, input_shape=(94,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_metric]) 
model.fit(X, y, epochs=100, batch_size=10)
# evaluate the keras model
accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))




Epoch 1/150
2979/2979 [==============================] - 13s 4ms/step - loss: -3080438.0000 - accuracy: 0.0562 - f1_metric: 0.7343
Epoch 2/150
2979/2979 [==============================] - 12s 4ms/step - loss: -31790494.0000 - accuracy: 0.0542 - f1_metric: 0.7358
Epoch 3/150
2979/2979 [==============================] - 11s 4ms/step - loss: -105543784.0000 - accuracy: 0.0542 - f1_metric: 0.7352
Epoch 4/150
2979/2979 [==============================] - 11s 4ms/step - loss: -237031488.0000 - accuracy: 0.0542 - f1_metric: 0.7361
Epoch 5/150
2979/2979 [==============================] - 11s 4ms/step - loss: -441022912.0000 - accuracy: 0.0542 - f1_metric: 0.7363
Epoch 6/150
2979/2979 [==============================] - 11s 4ms/step - loss: -729856000.0000 - accuracy: 0.0542 - f1_metric: 0.7357
Epoch 7/150
2979/2979 [==============================] - 10s 3ms/step - loss: -1115209472.0000 - accuracy: 0.0542 - f1_metric: 0.7363
Epoch 8/150
2979/2979 [==============================] - 11s 4ms/step -